In [1]:
import pandas as pd
from datetime import datetime
!python -V
import sys;
print(sys.version)

Python 3.9.5
3.9.5 (default, Jun  4 2021, 12:28:51) 
[GCC 7.5.0]


In [2]:
import numpy as np

#transactions_df = pd.read_pickle('dataset.pkl') #, index_col=0)
transactions_df = pd.read_pickle('dataset_sepa.pkl') #, index_col=0)

#transactions_df = transactions_df.drop("Unnamed: 0",axis=1)

colNames = {str(col): str(col).lower() for col in transactions_df.columns}
print(colNames)

transactions_df = transactions_df.rename(columns=colNames)
print(transactions_df.tx_datetime.head())

transactions_df.tx_datetime = pd.to_datetime(transactions_df.tx_datetime)

def date_to_timestamp(date_obj: datetime)-> int:
    return int(date_obj.timestamp() * 1000)

transactions_df.tx_datetime = transactions_df.tx_datetime.map(lambda x: date_to_timestamp(x))

#transactions_df.tx_datetime = transactions_df.tx_datetime.values.astype(np.int64)

print(transactions_df.tx_datetime.head())

#transactions_df.tx_datetime = transactions_df.tx_datetime.map(lambda x: transactions_df.to_timestamp(x))



{'TRANSACTION_ID': 'transaction_id', 'TX_DATETIME': 'tx_datetime', 'CUSTOMER_ID': 'customer_id', 'IBAN': 'iban', 'BIC': 'bic', 'FULLNAME': 'fullname', 'COUNTRY': 'country', 'EXT_CUSTOMER_ID': 'ext_customer_id', 'EXT_IBAN': 'ext_iban', 'EXT_BIC': 'ext_bic', 'EXT_NAME': 'ext_name', 'EXT_COUNTRY': 'ext_country', 'TX_OPERATION_TYPE': 'tx_operation_type', 'TX_DIRECTION': 'tx_direction', 'TX_STATUS': 'tx_status', 'TX_SEPA_SCHEME': 'tx_sepa_scheme', 'TX_CODE_STATUS': 'tx_code_status', 'TERMINAL_ID': 'terminal_id', 'TX_AMOUNT': 'tx_amount', 'TX_TIME_SECONDS': 'tx_time_seconds', 'TX_TIME_DAYS': 'tx_time_days', 'TX_FRAUD': 'tx_fraud', 'TX_FRAUD_SCENARIO': 'tx_fraud_scenario', 'TX_DURING_WEEKEND': 'tx_during_weekend', 'TX_DURING_NIGHT': 'tx_during_night', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW': 'customer_id_nb_tx_1day_window', 'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW': 'customer_id_avg_amount_1day_window', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW': 'customer_id_nb_tx_7day_window', 'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW':

In [3]:
import numpy as np

## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

df = reduce_mem_usage(transactions_df)  # attention ca prend pas le bon df en resultat transactions_df mais c'est fait dans la phase training 

Mem. usage decreased to 10.63 Mb (0.0% reduction)


In [4]:
fraud_labels = transactions_df.copy()[["tx_datetime","transaction_id", "tx_fraud"]]
fraud_labels #.read_csv('dataset_fraud.csv')

,tx_datetime,transaction_id,tx_fraud
0,1673654451000,695210,1
1,1673654494000,695211,0
2,1673654501000,695212,1
3,1673654611000,695213,0
4,1673654626000,695214,1
...,...,...,...
68832,1674259007000,764042,0
68833,1674259083000,764043,0
68834,1674259091000,764044,0
68835,1674259106000,764045,0


In [6]:
#%pip install dataframe_image lxml
import dataframe_image as dfi

df_recent = transactions_df.tail(5)
 
# If you exclude this image, you may have the same iris_latest.png and iris_actual.png files
# If no files have changed, the GH-action 'git commit/push' stage fails, failing your GH action (last step)
# This image, however, is always new, ensuring git commit/push will succeed.
dfi.export(df_recent, '../../assets/df_recent.png', table_conversion = 'matplotlib',max_cols=15)

In [ ]:
!pip install -U hopsworks --quiet

     |████████████████████████████████| 120 kB 13.3 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 132 kB 55.6 MB/s 
     |████████████████████████████████| 45 kB 2.7 MB/s 
     |████████████████████████████████| 68 kB 5.3 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 4.9 MB 57.8 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 2.8 MB 43.3 MB/s 
     |████████████████████████████████| 2.3 MB 48.4 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 109 kB 71.1 MB/s 
     |████████████████████████████████| 4.1 MB 46.9 MB/s 
     |████████████████████████████████| 140 kB 54.7 MB/s 
     |████████████████████████████████| 1.6 MB 54.1 MB/s 
     |████████████████████████████████| 127 kB 47.0 MB/s 
     |████████████████████████████████| 500 kB 55.6 MB/s 
     |████████████████████

In [ ]:
import hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3339
Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
trans_fg = fs.get_or_create_feature_group(
    name="trans_fraud",
    version=1,
    description="Transactions",
    primary_key=["transaction_id"],
    event_time = "tx_datetime"
)

In [ ]:
trans_fg.insert(transactions_df, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/3339/fs/3286/fg/4431


Uploading Dataframe: 0.00% |          | Rows 0/172860 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3339/jobs/named/trans_fraud_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f4fe359a490>, None)

In [ ]:
feature_descriptions = [
    {"name": "transaction_id", "description": "Transaction id"}
]

for desc in feature_descriptions: 
    trans_fg.update_feature_description(desc["name"], desc["description"])

In [ ]:
trans_label_fg = fs.get_or_create_feature_group(
    name="transactions_fraud_label",
    version=1,
    description="transactions that have been flagged as fraud",
    primary_key=['transaction_id'],
    event_time = "tx_datetime"
)

trans_label_fg.insert(fraud_labels, write_options={"wait_for_job" : False})

feature_descriptions = [
    {"name": "transaction_id", "description": "Transaction id"},
    {"name": "tx_fraud", "description": "Whether the transaction was fraudulent or not"}
]

for desc in feature_descriptions: 
    trans_label_fg.update_feature_description(desc["name"], desc["description"])

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/3339/fs/3286/fg/4432


Uploading Dataframe: 0.00% |          | Rows 0/172860 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3339/jobs/named/transactions_fraud_label_1_offline_fg_backfill/executions
